In [1]:
import pandas as pd
from typing import Optional, Dict, Any
import logging

def csv_to_parquet(
    csv_file: str,
    parquet_file: str,
    sep: str = ';',
    encoding: str = 'utf-8',
    dtype: Optional[Dict[str, Any]] = None
) -> None:
    """
    Convert a CSV file to Parquet format.
    
    Args:
        csv_file: Path to input CSV file
        parquet_file: Path to output Parquet file
        sep: Delimiter used in CSV (default semicolon)
        encoding: File encoding (default utf-8)
        dtype: Optional dictionary of column data types
    """
    try:
        # Read CSV with proper delimiter and encoding
        df = pd.read_csv(
            csv_file,
            sep=sep,
            quotechar='"',
            encoding=encoding,
            dtype=dtype
        )

        # Rename value column
        df.rename(columns={df.columns[3]: 'value'}, inplace=True)
        df.iloc[:, 3] = df.iloc[:, 3].astype('float64')
        df = df.drop(df.columns[0], axis=1)

        # Write to Parquet
        df.to_parquet(
            parquet_file,
            compression='snappy',
            index=False
        )
        
        # Verify the conversion
        df_verify = pd.read_parquet(parquet_file)
        print(f"Successfully converted {csv_file} to {parquet_file}")
        print(f"Number of rows: {len(df_verify)}")
        print(f"Columns: {df_verify.columns.tolist()}")
        
    except Exception as e:
        print(f"Error converting file: {str(e)}")
        raise

# Example usage
if __name__ == "__main__":
    files_to_convert = [181,188,191,192,193]
    for file in files_to_convert:
        csv_to_parquet(
            f'data/{file}.csv',
            f'data/{file}.parquet'
        )

Successfully converted data/181.csv to data/181.parquet
Number of rows: 738601
Columns: ['startTime', 'endTime', 'value']
Successfully converted data/188.csv to data/188.parquet
Number of rows: 743450
Columns: ['startTime', 'endTime', 'value']
Successfully converted data/191.csv to data/191.parquet
Number of rows: 743523
Columns: ['startTime', 'endTime', 'value']
Successfully converted data/192.csv to data/192.parquet
Number of rows: 742081
Columns: ['startTime', 'endTime', 'value']
Successfully converted data/193.csv to data/193.parquet
Number of rows: 742077
Columns: ['startTime', 'endTime', 'value']


In [2]:
import pandas as pd
df = pd.read_parquet('data/188.parquet')
print(df.columns)

Index(['startTime', 'endTime', 'value'], dtype='object')


In [3]:
import dlt
from dlt.sources.filesystem import filesystem, read_parquet

datasets = [
            ["181", "wind_energy_production"],
            ["188", "nuclear_energy_production"],
            ["191", "hydro_energy_production"],
            ["192", "total_energy_production"],
            ["193", "total_energy_consumption"]
           ]

for dataset in datasets:
    # Specify the exact file name
    files = filesystem(file_glob=f"{dataset[0]}.parquet")
    reader = (files | read_parquet()).with_name(dataset[1])

    pipeline = dlt.pipeline(pipeline_name="raw_csv_to_db_pipeline", dataset_name="raw_dataset", destination="motherduck")
    info = pipeline.run(reader, loader_file_format="parquet", write_disposition="merge", primary_key=("startTime", "endTime"))
    print(info)


TypeError: Pipeline.run() got an unexpected keyword argument 'full_refresh'

In [ ]:
import dlt
from dlt.sources.helpers import requests
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.auth import BearerTokenAuth

access_token = ""

# define new resource - github stargazers
@dlt.resource
def github_stargazers():
    client = RESTClient(
        base_url="https://api.github.com",
        auth=BearerTokenAuth(token=access_token)
    )

    for page in client.paginate("repos/dlt-hub/dlt/stargazers"):
        print(page)
        yield page


# define new dlt pipeline
pipeline = dlt.pipeline(destination="duckdb")


# run the pipeline with the new resource
load_info = pipeline.run(github_stargazers)
print(load_info)


# explore loaded data
pipeline.dataset(dataset_type="default").github_stargazers.df().shape

In [ ]:
import duckdb

# Connect to the database
conn = duckdb.connect("fingrid.duckdb")

# List all schemas
print("Schemas:")
print(conn.execute("SELECT schema_name FROM information_schema.schemata").df())

# List all tables in all schemas
print("\nTables:")
print(conn.execute("""
    SELECT table_schema, table_name 
    FROM information_schema.tables
    WHERE table_schema != 'information_schema'
""").df())

# Once you know the correct schema and table, query your data:
try:
    df = conn.execute("""
        SELECT * 
        FROM fingrid  -- replace with your actual schema.table name
        LIMIT 100
    """).df()
    print("\nFirst few rows:")
    print(df.head())
except Exception as e:
    print(f"Error querying table: {e}")

conn.close()

In [ ]:
import dlt
from dlt.sources.helpers import requests
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.auth import BearerTokenAuth

In [ ]:
api_key = ""

In [ ]:
import dlt
from dlt.sources.helpers import requests
from datetime import datetime, timedelta

# define dlt resources
@dlt.resource
def fingrid(dataset_id: str):
    client = RESTClient(
        base_url="https://data.fingrid.fi/api/",
        headers={
            "x-api-key": api_key,
            "Accept": "application/json"
        }
    )
    dataset_id = "181"
    start_time = (datetime.now() - timedelta(hours=1)).isoformat()
    end_time = datetime.now().isoformat()

    params = {
            "start_time": start_time,
            "end_time": end_time
        }
    endpoint = "datasets/181/data"
    response = client.get(endpoint, params=params)
    print(response.json()['data'])
    yield response.json()


# define dlt pipeline
pipeline = dlt.pipeline(destination="duckdb")

# run dlt pipeline
load_info = pipeline.run(fingrid("181"))
print(load_info)

# explore loaded data
pipeline.dataset(dataset_type="default").fingrid.df()

In [ ]:
import dlt
dlt.secrets["api_secret_key"]

In [ ]:
from typing import Any, Optional

import dlt
from dlt.common.pendulum import pendulum
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers import requests
from datetime import datetime, timedelta
from dlt.sources.rest_api import (
    RESTAPIConfig,
    check_connection,
    rest_api_resources,
    rest_api_source,
)


@dlt.source(name="fingrid")
def fingrid_source(api_key: Optional[str] = dlt.secrets.value) -> Any:
    # Create a REST API configuration for the GitHub API
    # Use RESTAPIConfig to get autocompletion and type checking
    client = RESTClient(
        base_url="https://data.fingrid.fi/api",
        headers={
            "x-api-key": api_key,
            "Accept": "application/json"
        }
    )

    datasets = [[181, "WindPower"]]
    start_time = (datetime.now() - timedelta(hours=1)).isoformat()
    end_time = (datetime.now()).isoformat()

    def get_resource(dataset):
        params = {
            "start_time": start_time,
            "end_time": end_time
        }
        endpoint = "/datasets/{dataset}/data"
        response = client.get(endpoint, params=params)
        yield response.json()

    for dataset in datasets:
        yield dlt.resource(get_resource(dataset[0]), name=dataset[1])



def load_fingrid() -> None:
    pipeline = dlt.pipeline(
        pipeline_name="rest_api_fingrid",
        destination='motherduck',
        dataset_name="rest_api_data",
    )

    load_info = pipeline.run(fingrid_source(dlt.secrets["api_secret_key"]))
    print(load_info)  # noqa: T201

if __name__ == "__main__":
    load_fingrid()
